In [28]:
!pip install plotly
!pip install folium

In [29]:
import pandas as pd
import plotly.express as px
import folium
from folium.plugins import HeatMap
import warnings
warnings.filterwarnings('ignore')

In [30]:
#@title Configuracion de colores
# Definición de paleta de colores personalizada
COLOR_PALETTE = {
    'primary': '#FF6B6B',    # Rojo coral
    'secondary': '#4ECDC4',  # Turquesa
    'accent1': '#45B7D1',    # Azul claro
    'accent2': '#96CEB4',    # Verde menta
    'accent3': '#FFEEAD',    # Amarillo pastel
}

In [31]:
df = pd.read_csv('/content/Presentacion de alumnos.csv')

In [33]:
#@title Ejecuta esta celda para ver la magia 🧙‍♂️
def load_data(file_path):
    """
    Carga y prepara los datos desde un archivo CSV
    """
    df = pd.read_csv(file_path)
    return df

def create_objective_pie_chart(df):
    """
    Crea un gráfico circular para los objetivos del curso
    """
    fig = px.pie(
        df,
        names='¿Cuál es tu objetivo principal al tomar este curso?',
        title='Objetivos principales al tomar el curso',
        color_discrete_sequence=list(COLOR_PALETTE.values())
    )
    fig.update_traces(textposition='inside', textinfo='percent+label')
    fig.update_layout(showlegend=False)
    return fig

def process_multiple_responses(df, column_name):
    """
    Procesa una columna que contiene múltiples respuestas separadas por punto y coma
    y retorna un DataFrame con el conteo de cada categoría
    """
    # Separar las respuestas y crear una lista plana
    all_responses = []
    for response in df[column_name]:
        categories = response.split(';')
        all_responses.extend(categories)

    # Contar las frecuencias
    response_counts = pd.Series(all_responses).value_counts()
    total_respondents = len(df)

    # Crear DataFrame con conteos y porcentajes
    result_df = pd.DataFrame({
        'Aspectos': response_counts.index,
        'Cantidad': response_counts.values,
        'Porcentaje': (response_counts.values / total_respondents * 100).round(1)
    })

    return result_df

def create_interests_bar_chart(df):
    """
    Crea un gráfico de barras horizontal para los aspectos interesantes
    procesando múltiples respuestas separadas por punto y coma
    """
    # Procesar los datos
    interests_df = process_multiple_responses(
        df,
        '¿Qué aspectos de la ciencia de datos te resultan más interesantes o emocionantes?'
    )

    # Crear el texto para las etiquetas
    interests_df['text'] = interests_df['Cantidad'].astype(str) + ' (' + \
                          interests_df['Porcentaje'].astype(str) + '%)'

    # Crear el gráfico
    fig = px.bar(
        interests_df,
        y='Aspectos',
        x='Porcentaje',
        orientation='h',
        text='text',
        title='¿Qué aspectos de la ciencia de datos te resultan más interesantes o emocionantes?'
    )

    # Personalizar el diseño
    fig.update_traces(
        marker_color='rgb(139, 69, 19)',  # Color marrón similar al de la imagen
        textposition='outside',
        textfont=dict(size=12),
    )

    fig.update_layout(
        plot_bgcolor='white',
        showlegend=False,
        xaxis=dict(
            title='',
            range=[0, 100],
            showgrid=True,
            gridcolor='rgba(0,0,0,0.1)',
            zeroline=True,
            zerolinecolor='rgba(0,0,0,0.1)',
            ticksuffix='%'
        ),
        yaxis=dict(
            title='',
            autorange="reversed"  # Para que el orden sea igual al de la imagen
        ),
        margin=dict(l=0, r=100, t=50, b=0),
        height=400
    )

    return fig

def create_age_bar_chart(df):
    """
    Crea un gráfico de barras vertical para rangos de edad
    """
    age_count = df['¿En que rango de edad estas?'].value_counts()

    fig = px.bar(
        age_count,
        title='Distribución por rango de edad',
        color_discrete_sequence=list(COLOR_PALETTE.values())
    )
    fig.update_layout(
        xaxis_title="Rango de edad",
        yaxis_title="Cantidad de estudiantes",
        showlegend=False
    )
    return fig

def create_programming_exp_pie(df):
    """
    Crea un gráfico circular para experiencia en programación
    """
    fig = px.pie(
        df,
        names='¿Has programado previamente?',
        title='Experiencia previa en programación',
        color_discrete_sequence=list(COLOR_PALETTE.values())
    )
    fig.update_traces(textposition='inside', textinfo='percent+label')
    fig.update_layout(showlegend=False)
    return fig

def create_country_map(df):
    """
    Crea un mapa interactivo con Folium mostrando la distribución por países
    """
    # Diccionario de coordenadas aproximadas para países latinoamericanos
    country_coords = {
        'Argentina': [-38.4161, -63.6167],
        'Chile': [-35.6751, -71.5430],
        'Peru': [-9.1900, -75.0152],
        'Colombia': [4.5709, -74.2973],
        'Mexico': [23.6345, -102.5528],
        # Agregar más países según sea necesario
    }

    # Crear el mapa base centrado en Sudamérica
    m = folium.Map(
        location=[-15, -60],
        zoom_start=3,
        tiles='CartoDB positron'
    )

    # Agregar marcadores para cada país
    for country in df['¿De que pais eres?'].unique():
        if country in country_coords:
            count = len(df[df['¿De que pais eres?'] == country])
            folium.CircleMarker(
                location=country_coords[country],
                radius=count * 5,  # Tamaño proporcional a la cantidad
                popup=f'{country}: {count} estudiantes',
                color=COLOR_PALETTE['primary'],
                fill=True,
                fill_color=COLOR_PALETTE['primary']
            ).add_to(m)

    return m

def main():
    """
    Función principal que ejecuta todo el pipeline de visualización
    """
    # Cargar datos
    # Para Google Colab, asegúrate de tener el archivo en tu directorio o usar la ruta correcta
    df = load_data('/content/Presentacion de alumnos.csv')

    # Crear visualizaciones
    objective_fig = create_objective_pie_chart(df)
    interests_fig = create_interests_bar_chart(df)
    age_fig = create_age_bar_chart(df)
    programming_fig = create_programming_exp_pie(df)
    country_map = create_country_map(df)

    # Mostrar las visualizaciones
    objective_fig.show()
    interests_fig.show()
    age_fig.show()
    programming_fig.show()
    display(country_map)

# Ejecutar todo
main()